#  Вложенные функции, замыкания и декораторы

## 1. Функции

**Функция** — часть программы, которую можно вызвать из другого места программы.

Все в Python объекты. И даже функции. Это значит, что у функций есть
- атрибуты
- и методы.

От остальных объектов функции отличаются тем, что их можно вызвать*. Объекты, которые можно вызвать, называют `Callable`-объектами. У них есть метод `__call__()`.

\* С точки зрения синтаксиса еще можно вызывать классы

### Как определить функцию

In [36]:
# Функция определяется таким синтаксисом
def plus_one(x: int) -> int:
    """Функция возвращает увеличенное на 1 целое число"""
    return x+1

Это избыточное определение. Из избыточного здесь использованы:
- строка документирования — `docsting`,
- и анотация функции.

На самом деле можно описать эту же функцию компактней. 

In [51]:
# Функция plus_one без анотаций и документации
def plus_one_simple(x): return x+1

### Функция как объект

Как у любого объекта в python, у функции есть:
- идентификатор,
- тип.

In [47]:
# У функции plus_one эти параметры выглядят так
id(plus_one), type(plus_one)

(4510970608, function)

В CPython идентификатор — **адрес объекта** в виртуальной памяти

In [48]:
# Идентификатор в шестнадцатиричном формате — адрес функции plus_one
hex(id(plus_one))

'0x10cdff2f0'

Все атрибуты и методы функции как объекта можно посмотреть:

In [45]:
import inspect
list(filter(lambda x: x[0] != "__globals__", sorted(inspect.getmembers(plus_one))))
# Здесь мы выбросили поле "__globals__", чтобы не засорять вывод

[('__annotations__', {'x': int, 'return': int}),
 ('__call__', <method-wrapper '__call__' of function object at 0x10cdff2f0>),
 ('__class__', function),
 ('__closure__', None),
 ('__code__',
  <code object plus_one at 0x10d59f780, file "<ipython-input-36-59ce0af9670a>", line 2>),
 ('__defaults__', None),
 ('__delattr__',
  <method-wrapper '__delattr__' of function object at 0x10cdff2f0>),
 ('__dict__', {}),
 ('__dir__', <function function.__dir__()>),
 ('__doc__', 'Функция возвращает увеличенное на 1 целое число'),
 ('__eq__', <method-wrapper '__eq__' of function object at 0x10cdff2f0>),
 ('__format__', <function function.__format__(format_spec, /)>),
 ('__ge__', <method-wrapper '__ge__' of function object at 0x10cdff2f0>),
 ('__get__', <method-wrapper '__get__' of function object at 0x10cdff2f0>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of function object at 0x10cdff2f0>),
 ('__gt__', <method-wrapper '__gt__' of function object at 0x10cdff2f0>),
 ('__hash__', <metho

### Как вызвать функцию

In [39]:
#  Вызов функции, ожидаем ответ 2
plus_one(1)

2

In [40]:
#  Можно явно вызвать метод call, ожидаем ответ 2
plus_one.__call__(1)

2

### Как функции устроены

In [41]:
# Байт-код функции function_name
plus_one.__code__.co_code

b'|\x00d\x01\x17\x00S\x00'

In [42]:
# Дизассемблированное тело функции function_name
import dis
dis.dis(plus_one)

  4           0 LOAD_FAST                0 (x)
              2 LOAD_CONST               1 (1)
              4 BINARY_ADD
              6 RETURN_VALUE


Если заглянуть во внутренности интерпретатора (CPython), то функция описывается следующей струтурой: https://github.com/python/cpython/blob/3.7/Include/funcobject.h

### Почитать
1. [The Python Language Reference. Data model](https://docs.python.org/3/reference/datamodel.html#objects-values-and-types)
2. [The Python Language Reference. Inspect live objects](https://docs.python.org/3/library/inspect.html)
3. [PEP 3107 -- Function Annotations](https://www.python.org/dev/peps/pep-3107/)
4. [PEP 257 -- Docstring Conventions](https://www.python.org/dev/peps/pep-0257/)

## 2. Вложенные функции

**Вложенная функция** — функция, которая определена внутри другой функции.

При работе с вложенными функциями надо учитывать области видимости.

### Область видимости в Python — LEGB

В Python есть 4 области видимости. Расположены они как показано на рисунке.

![title](img/LEGB.png)

Стрелки на рисунке показывают в какой последовательности Python обходит области видимости. Следующий код показывает как распределены области относительно вложенной функции inner.

In [64]:
# (built-in) — область системных имен

# global — область модуля
def outer():
    # enclosed — область функции-обёртки 
    def inner():
        # local — область внутри функции
        pass

### Зачем нужны вложенные функции?

Зачем это может быть нужно? Можно выделить 3 примера:
1. чтобы скрыть функцию в глобальной области видимости,
2. чтобы вынести «лишний» код из функцию в обёртку,
3. чтобы реализовать замыкания (см. следующий раздел).

#### Пример 1. Чтобы скрыть функцию — инкапсуляция

In [53]:
# Вложенная функция inner внутри plus_one_outer
def plus_one_outer(x: int) -> int:
    """Функция возвращает увеличенное на 1 целое число"""
    def inner(y: int) -> int: return y+1
    return inner(x)
    

In [55]:
#  Вызов функции, ожидаем ответ 2
plus_one_outer(1)

2

In [57]:
# Вложенная функция недоступна (должна быть ошибка)
inner(1)

NameError: name 'inner' is not defined

Вложенные функции дают накладные расходы

In [56]:
import dis
dis.dis(plus_one_outer)

  4           0 LOAD_GLOBAL              0 (int)
              2 LOAD_GLOBAL              0 (int)
              4 LOAD_CONST               1 (('y', 'return'))
              6 BUILD_CONST_KEY_MAP      2
              8 LOAD_CONST               2 (<code object inner at 0x10d59fc00, file "<ipython-input-53-07493295d00f>", line 4>)
             10 LOAD_CONST               3 ('plus_one_outer.<locals>.inner')
             12 MAKE_FUNCTION            4
             14 STORE_FAST               1 (inner)

  5          16 LOAD_FAST                1 (inner)
             18 LOAD_FAST                0 (x)
             20 CALL_FUNCTION            1
             22 RETURN_VALUE

Disassembly of <code object inner at 0x10d59fc00, file "<ipython-input-53-07493295d00f>", line 4>:
  4           0 LOAD_FAST                0 (y)
              2 LOAD_CONST               1 (1)
              4 BINARY_ADD
              6 RETURN_VALUE


#### Пример 2. Чтобы вынести «лишний» код из функции в обёртку

In [59]:
def factorial(x: int) -> int:
    """Функция вычисляет факториал целого числа"""
    def calc_factorial(y: int) -> int: return y * calc_factorial(y-1) if y!=0 else 1
    if x<0:
        return -1
    return calc_factorial(x)
    

In [62]:
factorial(4)

24

## 3. Замыкания

**Замыкание** — вложенная функция, которая запоминает значения окружения, с которым она была вызвана. Говорят, что функция «замыкается» на значения переменных окружения.

## 4. Декораторы

**Декоратор** — функция, которая принимает другую функцию и что-то возвращает.